# Embedding Constructs with a Sentence Transformer

In [60]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [61]:
# Loading data with pandas
neo_items =  pd.read_csv('NEO_items.csv', usecols=['construct', 'text'])
neo_items

,construct,text
0,Achievement-Striving,Go straight for the goal.
1,Achievement-Striving,Plunge into tasks with all my heart.
2,Achievement-Striving,Demand quality.
3,Achievement-Striving,Set high standards for myself and others.
4,Achievement-Striving,Turn plans into actions.
...,...,...
295,Vulnerability,Remain calm under pressure.
296,Vulnerability,Am calm even in tense situations.
297,Vulnerability,Can handle complex problems.
298,Vulnerability,Readily overcome setbacks.


In [62]:
# Embedding items
model = SentenceTransformer('all-MiniLM-L6-v2')
neo_items['embed'] = neo_items['text'].apply(lambda x: model.encode(x))
neo_items

,construct,text,embed
0,Achievement-Striving,Go straight for the goal.,"[0.010508136, 0.10021052, -0.07636007, 0.00171..."
1,Achievement-Striving,Plunge into tasks with all my heart.,"[-0.005888383, 0.01626125, 0.023178238, -0.011..."
2,Achievement-Striving,Demand quality.,"[-0.04246266, -0.0150403185, 0.018326316, -0.0..."
3,Achievement-Striving,Set high standards for myself and others.,"[0.0025991125, 0.051376447, -0.0050169597, -0...."
4,Achievement-Striving,Turn plans into actions.,"[-0.002227421, 0.07106716, 0.033379693, -0.022..."
...,...,...,...
295,Vulnerability,Remain calm under pressure.,"[0.008001904, 0.04168616, 0.028673416, 0.04836..."
296,Vulnerability,Am calm even in tense situations.,"[0.07779292, 0.017205391, 0.027962497, 0.01344..."
297,Vulnerability,Can handle complex problems.,"[-0.06403706, 0.10341669, 0.005948939, -0.0167..."
298,Vulnerability,Readily overcome setbacks.,"[-0.020695273, 0.061760057, 0.0056057484, 0.07..."


In [63]:
# Averaging item embeddings to get construct embeddings
construct_embeds = neo_items[['construct', 'embed']].groupby('construct').mean(numeric_only=False)
construct_embeds

,embed
construct,
Achievement-Striving,"[0.004298301, 0.03726425, 0.007912548, -0.0042..."
Activity-Level,"[0.001618525, -0.0006043544, 0.015233906, 0.04..."
Adventurousness,"[0.025800336, 0.004955708, 0.044256516, 0.0487..."
Aesthetic Appreciation/Artistic Interests,"[0.048290618, 0.025077041, 0.06637079, -0.0014..."
Altruism,"[0.0106899645, 0.026498854, 0.024884637, -0.01..."
Anger,"[0.018768145, 0.008189047, 0.044884227, 0.0493..."
Anxiety,"[0.03973216, 0.0023609183, 0.04619965, 0.05178..."
Assertiveness,"[0.026695859, 0.0009721123, -0.0056722173, -0...."
Cautiousness,"[0.03800477, -0.0043155407, 0.026376218, 0.019..."


In [64]:
# Reformatting the construct embeddings such that each column is an embedding dimension
list_of_embeds = construct_embeds['embed'].to_list()
construct_embeds = pd.DataFrame(list_of_embeds, index=construct_embeds.index)
construct_embeds

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
construct,,,,,,,,,,,,,,,,,,,,,
Achievement-Striving,0.004298,0.037264,0.007913,-0.004269,-0.018353,-0.024534,0.012682,-0.018506,-0.015681,0.005676,...,0.032268,0.011097,0.004869,0.020435,-0.064954,0.050593,0.072998,-0.023511,-0.039168,0.006319
Activity-Level,0.001619,-0.000604,0.015234,0.049282,0.046148,-0.005798,0.032762,-0.027327,0.021172,0.003219,...,0.037965,0.019592,0.011885,-0.003336,-0.004621,0.033700,0.038179,-0.020652,0.001250,-0.020256
Adventurousness,0.025800,0.004956,0.044257,0.048723,-0.011050,-0.011392,0.017286,-0.025266,0.005639,0.040548,...,0.059821,0.013491,0.006589,0.022432,0.010717,0.031726,0.047142,-0.007130,0.000165,0.012197
Aesthetic Appreciation/Artistic Interests,0.048291,0.025077,0.066371,-0.001419,-0.028676,0.025041,0.031712,-0.079248,0.031333,-0.000424,...,0.012982,-0.025234,-0.010336,0.068849,-0.009730,0.058120,0.048819,-0.013709,0.017838,-0.012106
Altruism,0.010690,0.026499,0.024885,-0.013526,0.021357,-0.020612,0.057289,-0.028740,0.009422,-0.016814,...,0.045150,-0.004136,0.000900,0.067190,0.022191,0.024426,0.049599,-0.025014,-0.042768,0.000707
Anger,0.018768,0.008189,0.044884,0.049368,0.015570,-0.012693,0.064483,-0.009928,0.017258,-0.020605,...,0.008850,-0.007560,0.015956,-0.000098,-0.000444,0.038712,0.040251,0.014108,-0.059225,-0.012340
Anxiety,0.039732,0.002361,0.046200,0.051786,0.043904,0.007185,0.056894,-0.015850,0.017888,-0.028865,...,0.035846,0.013569,0.005886,0.059854,0.018765,0.048199,0.037750,-0.008109,-0.062439,0.016752
Assertiveness,0.026696,0.000972,-0.005672,-0.019051,0.006382,0.015712,0.027128,-0.039620,0.019409,0.006405,...,0.065377,0.003177,0.016715,0.043915,-0.033603,0.061580,0.043831,-0.017892,-0.015792,0.022099
Cautiousness,0.038005,-0.004316,0.026376,0.019126,0.013720,0.006093,0.067304,-0.004575,0.031336,-0.002564,...,0.047820,-0.003146,-0.001774,0.028741,-0.026803,0.043275,0.047394,-0.030701,-0.044365,0.008522


# Comparing Predicted and Observed Construct Correlations

In [65]:
import numpy as np

In [66]:
# Calculating the correlation between construct embeddings
predicted = construct_embeds.T.corr()
predicted

construct,Achievement-Striving,Activity-Level,Adventurousness,Aesthetic Appreciation/Artistic Interests,Altruism,Anger,Anxiety,Assertiveness,Cautiousness,Cheerfulness,...,Liberalism,Modesty/Humility,Morality,Orderliness,Self-Discipline,Self-Efficacy,Self-consciousness,Sympathy,Trust,Vulnerability
construct,,,,,,,,,,,,,,,,,,,,,
Achievement-Striving,1.000000,0.683325,0.596813,0.438180,0.585849,0.477232,0.558175,0.705198,0.770884,0.520514,...,0.422136,0.610099,0.703681,0.656243,0.808163,0.751365,0.545400,0.532120,0.496445,0.629537
Activity-Level,0.683325,1.000000,0.642606,0.394938,0.572514,0.532134,0.572553,0.657762,0.713945,0.603842,...,0.363294,0.500476,0.563783,0.676124,0.732323,0.687178,0.536045,0.439350,0.428855,0.620304
Adventurousness,0.596813,0.642606,1.000000,0.564447,0.594343,0.467224,0.605937,0.596638,0.640601,0.497843,...,0.369372,0.542682,0.540330,0.701515,0.542919,0.622761,0.532884,0.474609,0.471584,0.511330
Aesthetic Appreciation/Artistic Interests,0.438180,0.394938,0.564447,1.000000,0.525627,0.392826,0.489925,0.517851,0.515103,0.482990,...,0.297742,0.479476,0.398306,0.523679,0.338669,0.432742,0.479937,0.460104,0.381838,0.422574
Altruism,0.585849,0.572514,0.594343,0.525627,1.000000,0.521328,0.629514,0.762620,0.592671,0.575594,...,0.446789,0.673930,0.712856,0.618876,0.504058,0.593737,0.669224,0.746703,0.730641,0.563196
Anger,0.477232,0.532134,0.467224,0.392826,0.521328,1.000000,0.642388,0.468607,0.497401,0.627919,...,0.243831,0.552908,0.482393,0.571992,0.459659,0.494562,0.572122,0.504789,0.450760,0.615453
Anxiety,0.558175,0.572553,0.605937,0.489925,0.629514,0.642388,1.000000,0.596294,0.677406,0.552312,...,0.290887,0.575119,0.550478,0.675768,0.587711,0.640205,0.775005,0.535626,0.495912,0.814823
Assertiveness,0.705198,0.657762,0.596638,0.517851,0.762620,0.468607,0.596294,1.000000,0.741140,0.541643,...,0.491006,0.682159,0.785709,0.645074,0.639355,0.703458,0.659220,0.636960,0.669129,0.630424
Cautiousness,0.770884,0.713945,0.640601,0.515103,0.592671,0.497401,0.677406,0.741140,1.000000,0.606092,...,0.421035,0.591183,0.696156,0.732107,0.729203,0.713076,0.584459,0.505587,0.533595,0.738444


In [67]:
# Loading observed correlations and pivoting to a correlation matrix
observed = pd.read_csv('NEO_correlations.csv')
observed = observed.pivot(index='construct_1', columns='construct_2', values='correlation')
observed

construct_2,Achievement-Striving,Activity-Level,Adventurousness,Aesthetic Appreciation/Artistic Interests,Altruism,Anger,Anxiety,Assertiveness,Cautiousness,Cheerfulness,...,Liberalism,Modesty/Humility,Morality,Orderliness,Self-Discipline,Self-Efficacy,Self-consciousness,Sympathy,Trust,Vulnerability
construct_1,,,,,,,,,,,,,,,,,,,,,
Achievement-Striving,1.000000,0.556188,0.207487,0.146712,0.297056,-0.169180,-0.182208,0.429419,0.355385,0.202051,...,-0.195316,-0.179229,0.245104,0.423729,0.692032,0.681050,-0.319778,0.039370,0.172744,-0.377009
Activity-Level,0.556188,1.000000,0.237717,0.056497,0.159652,0.011251,-0.081694,0.470846,0.066479,0.134995,...,-0.140695,-0.182681,0.021023,0.267310,0.526717,0.415360,-0.332381,-0.045992,0.061763,-0.233511
Adventurousness,0.207487,0.237717,1.000000,0.368639,0.222988,-0.246594,-0.391473,0.357362,-0.161081,0.323940,...,0.228079,-0.211193,-0.009990,-0.218838,0.112415,0.293686,-0.447935,0.130991,0.197621,-0.365071
Aesthetic Appreciation/Artistic Interests,0.146712,0.056497,0.368639,1.000000,0.375494,-0.094176,0.002154,0.131344,-0.003963,0.281623,...,0.248999,-0.011530,0.202955,-0.014465,0.040799,0.150727,-0.100406,0.390052,0.183400,-0.018475
Altruism,0.297056,0.159652,0.222988,0.375494,1.000000,-0.329779,-0.095435,0.176933,0.158027,0.464327,...,-0.012385,0.131117,0.491216,0.125416,0.237525,0.311276,-0.168456,0.630879,0.502078,-0.132680
Anger,-0.169180,0.011251,-0.246594,-0.094176,-0.329779,1.000000,0.623347,-0.030149,-0.303361,-0.295796,...,-0.000979,0.014338,-0.267615,-0.018533,-0.231005,-0.364793,0.302524,-0.165342,-0.395414,0.558404
Anxiety,-0.182208,-0.081694,-0.391473,0.002154,-0.095435,0.623347,1.000000,-0.278197,-0.110386,-0.340853,...,0.052482,0.222765,-0.023234,0.065618,-0.259856,-0.453877,0.596427,0.135380,-0.303660,0.792420
Assertiveness,0.429419,0.470846,0.357362,0.131344,0.176933,-0.030149,-0.278197,1.000000,-0.089866,0.395249,...,-0.066339,-0.565173,-0.205692,0.077129,0.297605,0.485758,-0.673060,-0.078673,0.135688,-0.379676
Cautiousness,0.355385,0.066479,-0.161081,-0.003963,0.158027,-0.303361,-0.110386,-0.089866,1.000000,-0.172144,...,-0.125071,0.113419,0.442165,0.433175,0.434872,0.444948,0.009633,0.117780,0.090374,-0.276091


In [68]:
# Aligning rows and columns the predicted and observed correlations
predicted, observed = predicted.align(observed)

def lower_triangle_flat(df):
    """Takes the lower triangle of a dataframe and flattens it into a vector"""
    rows, cols = np.triu_indices(len(df), k=1)
    return pd.Series(df.values[rows, cols])

predicted, observed = lower_triangle_flat(predicted), lower_triangle_flat(observed)

# Correlation between predicted and observed
print(f'r: {predicted.corr(observed).round(2)}')
print(f'r of absolute values: {predicted.abs().corr(observed.abs()).round(2)}')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
r: 0.24
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
r of absolute values: 0.5
